In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import requests 
import json
from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from lxml import etree

In [6]:
def extract(page_source, start_sep, end_sep):
    source=page_source

    result=[]
    tmp=source.split(start_sep)
    for par in tmp:
        if end_sep in par:
            result.append(par.split(end_sep)[0])
    return result

def extract2(string, start, end):
    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [3]:
def empty_item():
    global product_details
    product_details = {
        'name': None,
        'sku': None,
        'gtin': None,
        'color': None,
        'finish': None,
        'price': None,
        'product_description': None,
        'technical_description': None,
        'display_dimensions': None,
        'length': None,
        'width': None,
        'height': None,
        'depth': None,
        'diameter': None,
        'weight': None,
        'arm_height': None,
        'seat_height': None,
        'table_clearance_height': None,
        'maximum_dimensions': None,
        'hero_image': None,
        'variant_image': None,
        'images': None,
        'files': None,
        'installations': None,
        'designer': None,
        'collection': None,
        'categories': None,
        'available_new': None,
        'made_in': None,
        'voltage': None,
        'bulb_type': None,
        'bulb_text': None,
        'canopy_dimensions': None,
        'ul_listed': None,
        'certifications': None,
        'warranty_&_care': None,
        'position': None,
        'notes': None,
        'url':  None
    }
    return product_details




In [95]:
'arjun'[:-1]

'arju'

In [96]:
chrome_options = Options()
headless = False

if headless:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")


prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
#         chrome_path = which("chromedriver_win32/chromedriver")
global driver
driver = webdriver.Chrome(executable_path=r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe", options=chrome_options)
driver.get("https://innitdesigns.com/collections/frame")

time.sleep(1)

product_url = driver.find_elements(By.XPATH, "//div[@class = 'grid-view-item product-card']")
items = []
for product in product_url:
    empty_item()
    
    name_path = product.find_element(By.XPATH, ".//div[2]")
    product_details['name'] = name_path.text

    href = product.find_element(By.XPATH, ".//a").get_attribute("href")
    driver.execute_script("window.open('" +href +"');")
    print(href)
    product_details['url'] = href
#     switch to new window
    driver.switch_to.window(driver.window_handles[1])
    
    try:
        tech_desc = driver.find_elements(By.XPATH, "//div[@class = 'desc_product']/div/div")
        technical_description = []
        for tech in tech_desc:
            heading = tech.find_element(By.XPATH, ".//p")
            technical_description.append(heading.text.rstrip().lstrip())
            content = tech.find_elements(By.XPATH, ".//ul/li")
            for con in content:
                technical_description.append(con.text.rstrip().lstrip())

        product_details['technical_description'] = '\n'.join(technical_description) 
    except:
        product_details['technical_description'] = None
    
    
    
    response = requests.get(href)
    page_source = response.text
    
    desc = extract(page_source, '<script type="application/ld+json">', '</script>' )
    desc = json.loads(desc[-1])
    
    description = desc['description']
    product_details['product_description'] = description
    
    info = extract(page_source, '<script type="application/json" id="ProductJson-product-template">', '</script>' )
    info = json.loads(info[1])
    
    hero_image = info['featured_image']
    product_details['hero_image'] =  'https:' + hero_image
    
    images = info['images']
    images = ['https:' + img for img in images]
    product_details['images'] = '\n'.join(images)
    
    option_names = info['options']
    option_names = [opt[:-1].lower() for opt in option_names]
    variants = info['variants']
    for var in variants:
        if 'Select' in var['title']:
            continue
        product_details['sku'] = var['sku']
        product_details['product_id'] = var['id']
        product_details[f'option({option_names[0]})'] = var['option1']
        try:
            product_details[f'option({option_names[1]})'] = var['option2']
        except:
            product_details['option2'] = var['option2']

        product_details['price'] = var['price']/100
        product_details['compare_at_price'] = var['compare_at_price']
        product_details['available_new'] = var['available']
        
        try:
            product_details['variant_image'] = var['featured_image']['src']
        except:
            product_details['variant_image'] = None
        
        items.append(product_details.copy())

#         print(product_details)

    

    driver.close()

    #back to main window
    driver.switch_to.window(driver.window_handles[0])


C:\Users\kchai\AppData\Local\Temp\ipykernel_14056\3020593647.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe", options=chrome_options)


https://innitdesigns.com/collections/frame/products/acapulco-chair
https://innitdesigns.com/collections/frame/products/acapulco-colorful-frames
https://innitdesigns.com/collections/frame/products/multicolor-acapulco-chairs
https://innitdesigns.com/collections/frame/products/acapulco-chair-palm-leaf
https://innitdesigns.com/collections/frame/products/leather-acapulco-chair
https://innitdesigns.com/collections/frame/products/innit-chair
https://innitdesigns.com/collections/frame/products/junior-acapulco
https://innitdesigns.com/collections/frame/products/acapulco-rocker
https://innitdesigns.com/collections/frame/products/junior-acapulco-rocker
https://innitdesigns.com/collections/frame/products/micro-acapulco
https://innitdesigns.com/collections/frame/products/innit-rocker
https://innitdesigns.com/collections/frame/products/concha-chair
https://innitdesigns.com/collections/frame/products/puerto-stool-bar-height
https://innitdesigns.com/collections/frame/products/puerto-stool-counter-heig

In [97]:
len(items)

1379

In [98]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df


,name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,length,...,notes,url,product_id,option(weave color),option(frame color),compare_at_price,option(titl),option2,option(sizeweave color),option(colo)
0,Acapulco Chair,,None,None,None,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,39381368144080,Black,Black,None,NaN,NaN,NaN,NaN
1,Acapulco Chair,,None,None,None,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,39381368176848,Black,White,None,NaN,NaN,NaN,NaN
2,Acapulco Chair,,None,None,None,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,39381368209616,Black,Chrome,None,NaN,NaN,NaN,NaN
3,Acapulco Chair,,None,None,None,480.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,39381368242384,Black,Copper,None,NaN,NaN,NaN,NaN
4,Acapulco Chair,,None,None,None,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,39381368307920,White,Black,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,Vinyl Cord - 1000 foot rolls,,None,None,None,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42175244730576,NaN,NaN,NaN,NaN,NaN,NaN,Coral
1375,Vinyl Cord - 1000 foot rolls,,None,None,None,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42212315300048,NaN,NaN,NaN,NaN,NaN,NaN,Silver
1376,Vinyl Cord - 1000 foot rolls,,None,None,None,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42212315693264,NaN,NaN,NaN,NaN,NaN,NaN,Clear with Sparkles
1377,Vinyl Cord - 1000 foot rolls,,None,None,None,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42212316184784,NaN,NaN,NaN,NaN,NaN,NaN,Pink with Sparkles


In [99]:
df = df.replace({np.nan: None, '': None})


In [100]:
df['technical_description'].isna().sum()

504

In [101]:
df[df['name'] == 'Vinyl Cord - 135 foot rolls']

,name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,length,...,notes,url,product_id,option(weave color),option(frame color),compare_at_price,option(titl),option2,option(sizeweave color),option(colo)
1332,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076267216,None,None,None,None,None,None,Black (gloss)
1333,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076332752,None,None,None,None,None,None,White
1334,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076398288,None,None,None,None,None,None,Yellow
1335,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076431056,None,None,None,None,None,None,Blue
1336,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076463824,None,None,None,None,None,None,Pink
1337,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42212348690640,None,None,None,None,None,None,Grey
1338,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076496592,None,None,None,None,None,None,Purple
1339,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076529360,None,None,None,None,None,None,Red
1340,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076562128,None,None,None,None,None,None,Turquoise
1341,Vinyl Cord - 135 foot rolls,None,None,None,None,35.65,SHIPS WITHIN 7 DAYS\nThis is the vinyl cording...,None,None,None,...,None,https://innitdesigns.com/collections/frame/pro...,42061076594896,None,None,None,None,None,None,Orange


In [127]:
df['technical_description']

0       Additional Info\nApproved for commercial use (...
1       Additional Info\nApproved for commercial use (...
2       Additional Info\nApproved for commercial use (...
3       Additional Info\nApproved for commercial use (...
4       Additional Info\nApproved for commercial use (...
                              ...                        
1374                                                  NaN
1375                                                  NaN
1376                                                  NaN
1377                                                  NaN
1378                                                  NaN
Name: technical_description, Length: 1379, dtype: object

In [116]:
df = pd.read_excel(r'C:\Users\kchai\Downloads\innitdesigns.xlsx')
df

,name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,length,...,ul_listed,certifications,warranty_&_care,position,notes,url,product_id,option(weave color),option(frame color),option(color)
0,Acapulco Chair,NaN,NaN,NaN,NaN,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,39381368144080,Black,Black,NaN
1,Acapulco Chair,NaN,NaN,NaN,NaN,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,39381368176848,Black,White,NaN
2,Acapulco Chair,NaN,NaN,NaN,NaN,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,39381368209616,Black,Chrome,NaN
3,Acapulco Chair,NaN,NaN,NaN,NaN,480.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,39381368242384,Black,Copper,NaN
4,Acapulco Chair,NaN,NaN,NaN,NaN,460.0,SHIPS WITHIN 7 DAYS\n30” (W) x 36”(D) x 36”(H)...,Additional Info\nApproved for commercial use (...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,39381368307920,White,Black,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,Vinyl Cord - 1000 foot rolls,NaN,NaN,NaN,NaN,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,42175244730576,NaN,NaN,Coral
1375,Vinyl Cord - 1000 foot rolls,NaN,NaN,NaN,NaN,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,42212315300048,NaN,NaN,Silver
1376,Vinyl Cord - 1000 foot rolls,NaN,NaN,NaN,NaN,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,42212315693264,NaN,NaN,Clear with Sparkles
1377,Vinyl Cord - 1000 foot rolls,NaN,NaN,NaN,NaN,207.0,"SHIPS WITHIN 7 DAYS\nVinyl 0.2 inch / 5mm, mad...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://innitdesigns.com/collections/frame/pro...,42212316184784,NaN,NaN,Pink with Sparkles


In [123]:
df['shipping_time'] = None
for i in range(len(df)):
    df['product_description'][i] = df['product_description'][i].replace('\xa0', '')
    if 'SHIPS' in df['product_description'][i]:
        temp = df['product_description'][i].split('\n')
        df['shipping_time'][i] =  temp[0]
        df['product_description'][i] = '\n'.join(temp[1:])


C:\Users\kchai\AppData\Local\Temp\ipykernel_14056\2101246023.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_description'][i] = df['product_description'][i].replace('\xa0', '')
C:\Users\kchai\AppData\Local\Temp\ipykernel_14056\2101246023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['shipping_time'][i] =  temp[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_14056\2101246023.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [133]:
for i in range(len(df)):
    try:
        if 'Additional Info' in df['technical_description'][i]:
            temp = df['technical_description'][i].split('\n')
            df['technical_description'][i] = '\n'.join(temp[1:])
#             print(i)
    except:
        pass

C:\Users\kchai\AppData\Local\Temp\ipykernel_14056\1868351770.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['technical_description'][i] = '\n'.join(temp[1:])


In [134]:
df['technical_description'][40]

'Approved for commercial use (BIFMA)\nStackable (5 high)\nWeatherproof\nBreathable\nEasy to clean & doesn’t stay wet\nWill never blow over\nCool as breeze passes over it\nSeat does not get hot in the sun (not even the black)\nMay be easily disassembled for convenient storage and transport\nVinyl:\nChairs may be woven in multiple colors and patterns\nCustom colors may be ordered for an additional $500 (unlimited quantity)\nProtected against UV rays and mould\n18 Available colors\nFrames:\nStainless steel frames available as custom order\nCustom frame finish may be ordered for an additional $500 (unlimited quantity)\nFrames are e-coated to increase lifespan\nPowder coated frames are suitable for outdoor use (chrome and copper for indoor)'

In [135]:
df.to_excel('innitdesigns.xlsx', index = False)

In [124]:
df['product_description'][1343]

"This is the vinyl cording used to make our Acapulco chairs and other woven indoor / outdoor chairs. This is incredibly strong stuff with UV protection to keep the material from degrading or losing it's colour, also with anti-fungal protection. It is flexible but not so much that it will stretch out. Available in 18 colors here on 135 foot rolls. Made in Canada using organic plasticizers. The greenest flexible PVC on the market. Do you have some vintage furniture to rewrap? Did you buy a cheaply made Acapulco chair style from a discount shop or the knock of store and the material broke on you? We have been making furniture out of very strong vinyl cord since 2009 and we have not had even one complaint of the material breaking on its own!It takes about 2 - 2.5 rolls to wrap almost any full size chair. We usually recommend buying a bit extra to avoid the frustration of working on a chair and finding you don't have enough to finish.\nVinyl 0.2 inch / 5mm, made in Canada, PBA-Free, flexibl

In [73]:
href = 'https://innitdesigns.com/collections/frame/products/vinyl-cord-135-foot-rolls'
response = requests.get(href)
page_source = response.text
info = extract(page_source, '<script type="application/json" id="ProductJson-product-template">', '</script>' )
info = json.loads(info[1])
info

{'id': 7330943467728,
 'title': 'Vinyl Cord - 135 foot rolls',
 'handle': 'vinyl-cord-135-foot-rolls',
 'description': '<p><span>SHIPS WITHIN 7 DAYS</span></p>\n<p>This is the vinyl cording used to make our <a href="https://innitdesigns.com/collections/frame/products/acapulco-chair" target="_blank" title="Acapulco Chair" rel="noopener noreferrer">Acapulco chairs</a> and other woven indoor / outdoor chairs. This is incredibly strong stuff with UV protection to keep the material from degrading or losing it\'s colour, also with anti-fungal protection. It is flexible but not so much that it will stretch out. Available in 18 colors here on 135 foot rolls. Made in Canada using organic plasticizers. The greenest flexible PVC on the market. <br><br>Do you have some vintage furniture to rewrap? Did you buy a cheaply made Acapulco chair style from a discount shop or the knock of store and the material broke on you? We have been making furniture out of very strong vinyl cord since 2009 and we hav

In [74]:
variants = info['variants']

In [75]:
info['options']

['Color']

In [76]:
variants

[{'id': 42061076267216,
  'title': 'Black (gloss)',
  'option1': 'Black (gloss)',
  'option2': None,
  'option3': None,
  'sku': '',
  'requires_shipping': True,
  'taxable': True,
  'featured_image': {'id': 30150976438480,
   'product_id': 7330943467728,
   'position': 2,
   'created_at': '2022-03-07T14:09:39-05:00',
   'updated_at': '2022-03-07T14:12:55-05:00',
   'alt': None,
   'width': 5213,
   'height': 3476,
   'src': 'https://cdn.shopify.com/s/files/1/0555/6925/2560/products/glossblack2018-11-2014.58.28.jpg?v=1646680375',
   'variant_ids': [42061076267216]},
  'available': True,
  'name': 'Vinyl Cord - 135 foot rolls - Black (gloss)',
  'public_title': 'Black (gloss)',
  'options': ['Black (gloss)'],
  'price': 3565,
  'weight': 907,
  'compare_at_price': None,
  'inventory_management': 'shopify',
  'barcode': '',
  'featured_media': {'alt': None,
   'id': 22422305177808,
   'position': 2,
   'preview_image': {'aspect_ratio': 1.5,
    'height': 3476,
    'width': 5213,
    'src

In [77]:
i = 0
for var in variants:
    if 'Select' in var['title']:
        continue
    print(var['sku'])
    print(var['id'])
    print(var['available'])
    print(var['option1'])
    print(var['option2'])
    print(var['option3'])
    print(var['price'])
    print(var['compare_at_price'])
    print(var['featured_image']['src'])
    i += 1



42061076267216
True
Black (gloss)
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/glossblack2018-11-2014.58.28.jpg?v=1646680375

42061076332752
True
White
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/white2018-11-2014.57.43.jpg?v=1646680375

42061076398288
True
Yellow
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/yellow2018-11-2014.52.23.jpg?v=1646680375

42061076431056
True
Blue
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/blue2018-11-2014.58.11copy.jpg?v=1646680375

42061076463824
True
Pink
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/pink2018-11-2014.53.03.jpg?v=1646680375

42212348690640
True
Grey
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/download_1.png?v=1646680375

42061076496592
True
Purple
None
None
3565
None
https://cdn.shopify.com/s/files/1/0555/6925/2560/products/innitswatchpurple

In [45]:
i

0

In [24]:
desc = extract(page_source, '<script type="application/ld+json">', '</script>' )
desc = json.loads(desc[-1])
desc


{'@context': 'http://schema.org/',
 '@type': 'Product',
 'name': 'Puerto Stool - Counter Height',
 'url': 'https://innitdesigns.com/products/puerto-stool-counter-height',
 'image': ['https://cdn.shopify.com/s/files/1/0555/6925/2560/products/i10b-04-27q_d11e1b56-fabf-4778-ac07-5ff2b8b5f828_3600x.jpg?v=1647356895'],
 'description': 'SHIPS WITHIN 7 DAYS\nCounter Height Dimensions\n25” (W) x 20”(D) x 40”(H) x 26”(SH) (20 lbs)64 cm(W) x 51 cm(D) x 114 cm(H) x 66 cm (SH) (9 k)\n\xa0\nThe Puerto Stool first debuted at the chic Standard Hotel in Miami and became so popular it’s become a permanent addition to our collection. Inspired by our own Concha dining chair, it’s available in both bar or counter height and looks stylish in both residential and commercial settings. Comfortable for cocktails and beyond, it’s easy to clean and - for outdoor living - is designed to never blow over and dry quickly when wet. Available in 18 shades of colorfast UV-resistant woven vinyl cord and 4 durable frame 

In [25]:
desc['description']

'SHIPS WITHIN 7 DAYS\nCounter Height Dimensions\n25” (W) x 20”(D) x 40”(H) x 26”(SH) (20 lbs)64 cm(W) x 51 cm(D) x 114 cm(H) x 66 cm (SH) (9 k)\n\xa0\nThe Puerto Stool first debuted at the chic Standard Hotel in Miami and became so popular it’s become a permanent addition to our collection. Inspired by our own Concha dining chair, it’s available in both bar or counter height and looks stylish in both residential and commercial settings. Comfortable for cocktails and beyond, it’s easy to clean and - for outdoor living - is designed to never blow over and dry quickly when wet. Available in 18 shades of colorfast UV-resistant woven vinyl cord and 4 durable frame finishes. Note: Chrome and copper frame finishes are suitable for indoor use only. Proudly handmade in Toronto. Please allow two weeks for delivery to the United States and Canada.'